In [25]:
from tqdm import tqdm_notebook as tqdm
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import brown

# read manually curated csv

In [15]:
df = pd.read_csv('data/microsoft/to_be_checked.csv', encoding='ISO-8859-1')[['Word', 'Replacement']].dropna()

In [18]:
df.sample(5)

,Word,Replacement
413,EXPLODE,blow up
324,ELICIT,"draw out, prompt"
219,CONTRA,against
606,NARY,not
355,ENSURES,makes sure


In [22]:
dc = {}
for idx, row in df.iterrows():
    repl = [x.strip() for x in row['Replacement'].split(',')]
    dc[row['Word'].lower()] = repl
dc

{'abaft': ['behind'],
 'abeyance': ['suspension'],
 'abominate': ['hate'],
 'abundance': ['lot'],
 'accelerated': ['sped up'],
 'accessibility': ['ease of use'],
 'accompanied by': ['with'],
 'accompanies': ['comes with'],
 'accompany': ['come with'],
 'accompanying': ['coming with'],
 'accordingly': ['so'],
 'accrue': ['gather', 'gain'],
 'accrued': ['gathered', 'gained'],
 'accrues': ['gathers', 'gains'],
 'accruing': ['gathering', 'gaining'],
 'accurate': ['correct', 'right'],
 'acknowledge': ['noted'],
 'acknowledged': ['noted'],
 'acknowledges': ['notes'],
 'acknowledging': ['noting'],
 'acknowledgment': ['notice'],
 'acquiesce': ['accept', 'allow'],
 'acquisition': ['acquiring'],
 'activated': ['started'],
 'activation': ['start'],
 'additional': ['more'],
 'adjustment': ['change'],
 'adjustments': ['changes'],
 'administration': ['people in charge'],
 'administrative': ['managing'],
 'administrator': ['manager', 'person in charge'],
 'admissible': ['allowed'],
 'aforementioned':

In [23]:
len(dc)

613

In [ ]:
# oanc_df with 'text' 'label'

In [ ]:
path = '/home/rebekah/Documents/BNC/Texts/'
files = [f for f in glob(path + "**/*.xml", recursive=True)]

In [ ]:
skip = [['Voice', 'over'], ['Male', 'speaker'], ['Female', 'speaker']]

def parse_bnc_xml(path):
    # takes path to BNC xml file
    # returns description of texts
    # returns list of documents, each of which is a list of sentences, each of which is a list of tokens
    
    tree = ET.parse(path)
    root = tree.getroot()
    info = root[0][0][0][0].text
    documents = []
    
    for div in root[1]:
        doc = []
        sent = []
        for tag in div.iter():
            if tag.text == '\n':
                sent = list(filter(None, sent))
                if sent.count('.') > 1:
                    temp = ' '.join(sent)
                    temp = sent_tokenize(temp)
                    for temp_sent in temp:
                        temp_sent = word_tokenize(temp_sent)
                        if len(sent) < 2:
                            sent = []
                        elif temp_sent in skip: #skip certain things
                            sent = []
                        elif any([piece.isupper() for piece in temp_sent]):
                            sent = []
                        else:
                            doc.append(temp_sent)
                elif len(sent) > 1:
                    if sent in skip:
                        sent = []
                    elif any([piece.isupper() for piece in sent]):
                        sent = []
                    else:
                        doc.append(sent)
                sent = []
            elif tag.text != None:
                sent.append(tag.text.strip())
        if len(doc) > 1:
            documents.append(doc)
    
    return info, documents

In [ ]:
label = []
sent1 = []
sent2 = []
sent2_og = []
y = []

for file in tqdm(files):
    info, docs = parse_bnc_xml(file)
    for doc in docs:
        for idx in range(len(doc))[1:]:
            found = False
            for word in doc[idx][:3]:
                if word in terms_dict:
                    clean = clean_sent(doc[idx], word)
                    if type(clean) != list:
                        break
                    label.append(info)
                    sent1.append(doc[idx-1])
                    sent2.append(clean)
                    sent2_og.append(doc[idx])
                    this_y = np.zeros(len(terms_dict) + 1)
                    this_y[terms_dict[word]] = 1
                    y.append(this_y)
                    found = True
            if not found:
                label.append(info)
                sent1.append(doc[idx-1])
                sent2.append(doc[idx])
                sent2_og.append(None)
                this_y = np.zeros(len(terms_dict) + 1)
                this_y[9] = 1
                y.append(this_y)
        break
                
df = pd.DataFrame()
df['label'] = label
df['sent1'] = sent1
df['sent2'] = sent2
df['sent2_orig'] = sent2_og 
df['y'] = y

In [ ]:
sents = brown.sents()